**모델, 토크나이저 불러오기**

In [21]:
import json
from keras_crf import CRFModel
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 모델을 저장하고 불러오기 위해 정의
class CustomCRFModel(CRFModel):
    def __init__(self, model, units):
        super(CustomCRFModel, self).__init__(model, units)
        self.model = model
        self.units = units

    def get_config(self):
        config = {
            'model': self.model.get_config(),
            'units': self.units
        }
        return config

    @classmethod
    def from_config(cls, config):
        model = Model.from_config(config['model'])
        units = config['units']
        return cls(model, units)

custom_objects = {
    'CustomCRFModel': CustomCRFModel
}

# 저장된 모델 불러오기
model = load_model('bilstm_crf.h5', custom_objects=custom_objects, compile=False)
model.compile(optimizer=Adam(0.001), metrics='accuracy')

# 저장된 토크나이저 불러오기
with open('sentence_tokenizer_json.json', 'r', encoding='utf-8') as f:
    sentence_tokenizer_json = json.load(f)
    sentence_tokenizer = tokenizer_from_json(sentence_tokenizer_json)

with open('tag_tokenizer_json.json', 'r', encoding='utf-8') as f:
    tag_tokenizer_json = json.load(f)
    tag_tokenizer = tokenizer_from_json(tag_tokenizer_json)

In [9]:
model.summary()

Model: "custom_crf_model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 70)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 70, 128)      45852928    ['sequence_input[0][0]']         
                                                                                                  
 bidirectional (Bidirectional)  (None, 70, 128)      98816       ['embedding[0][0]']              
                                                                                                  
 time_distributed (TimeDistribu  (None, 70, 128)     0           ['bidirectional[0][0]']          
 ted)                                                                            

In [20]:
# text = '태안군의회 김기두 의장이 지난해 거둔 성과를 바탕으로 올 한해 군민 중심의 의정활동을 펼치겠다는 힘찬 포부를 밝혔다.'
import re
text = "올 가을 출시 예정인 애플의 아이폰16 시리즈 모든 기종에 A18 칩이 탑재될 것으로 보인다."

def predict(text):
    text = re.sub('[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', " ", text)
    text = text.split(" ")
    print(text)

    index_to_tag = tag_tokenizer.index_word

    X = sentence_tokenizer.texts_to_sequences([text])

    max_len = 70
    X = pad_sequences(X, padding='post', maxlen=max_len)

    y_predicted = model.predict(X)[0]

    print("단어                 예측값")
    print("-" * 35)

    for i, pred in enumerate(y_predicted[0]):
        if pred != 0: # PAD값은 제외
            print(f"{text[i]:17} {index_to_tag[pred]}")

predict(text)

['올', '가을', '출시', '예정인', '애플의', '아이폰16', '시리즈', '모든', '기종에', 'A18', '칩이', '탑재될', '것으로', '보인다.']
1/1 [==============================] - 0s 63ms/step
단어                 예측값
-----------------------------------
올                 B-DT
가을                I-DT
출시                O
예정인               O
애플의               O
아이폰16             O
시리즈               O
모든                O
기종에               O
A18               O
칩이                O
탑재될               O
것으로               O
보인다.              O
